In [ ]:
import tkinter as tk
from tkinter import messagebox,simpledialog
from PIL import Image, ImageTk
import cv2
import os
import datetime

import numpy as np
import face_recognition
        
from datetime import datetime
        
from openpyxl import Workbook, load_workbook
from test import test
faceDetect=cv2.CascadeClassifier("haarcascade_frontalface_default.xml");
class RegisterApp:
    def __init__(self, window):
        self.window = window
        self.main_window = None  
        self.window.title("User Registration")

        self.video_source = 0
        self.cap = cv2.VideoCapture(self.video_source)
        self.canvas = tk.Canvas(window, width=640, height=480)
        self.canvas.pack()

        self.btn_capture = tk.Button(window, text="Capture Photo", command=self.capture_photo)
        self.btn_capture.pack(pady=10)

        self.btn_save = tk.Button(window, text="Save Photo", command=self.save_photo)
        self.btn_save.pack(pady=10)

        

        self.btn_back = tk.Button(window, text="Back", command=self.go_back)
        self.btn_back.pack(pady=10)

        self.delay = 10
        self.update()

    def capture_photo(self):
        if hasattr(self, 'captured_photo'):
            del self.captured_photo  

        ret, frame = self.cap.read()
        if ret:
            self.captured_photo = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            self.photo = ImageTk.PhotoImage(self.captured_photo)
            self.canvas.create_image(0, 0, image=self.photo, anchor=tk.NW)
        messagebox.showinfo("Success", "Photo captured successfully!")

    def save_photo(self):
        if hasattr(self, 'captured_photo'):
            name = tk.simpledialog.askstring("Enter Name", "Enter the name to save the photo:")
            if name:
                save_path = "C:\\Users\\hp\\Desktop\\pbl"  

                
                if not os.path.exists(save_path):
                    os.makedirs(save_path)

               
                filename = f"{name}.jpg"

                
                self.captured_photo.save(os.path.join(save_path, filename))

                
                messagebox.showinfo("Success", "Photo saved successfully!")
            else:
                messagebox.showwarning("No Name", "Please enter a name!")
        else:
            messagebox.showwarning("No Photo", "Capture a photo first!")
        del self.captured_photo
    

    def update(self):
        ret, frame = self.cap.read()
        if ret:
            self.photo = ImageTk.PhotoImage(image=Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
            self.canvas.create_image(0, 0, image=self.photo, anchor=tk.NW)

        self.window.after(self.delay, self.update)

    def go_back(self):
        self.window.destroy()
        self.main_window.deiconify()

class WelcomeScreen:
    def __init__(self, window):
        self.window = window
        self.window.title("Welcome")
        
        self.label = tk.Label(window, text="Welcome to Face Recognition based Attendance System", font=("Arial", 16))
        self.label.pack(pady=20)

        self.btn_registration = tk.Button(window, text="User Registration", command=self.open_registration_window)
        self.btn_registration.pack(pady=10)

        self.btn_attendance = tk.Button(window, text="Attendance", command=self.start_attendance)
        self.btn_attendance.pack(pady=10)

    def open_registration_window(self):
        self.window.withdraw()  
        register_window = tk.Toplevel(self.window)
        app = RegisterApp(register_window)
        app.main_window = self.window 
        register_window.protocol("WM_DELETE_WINDOW", self.handle_registration_close)

    def handle_registration_close(self):
        if messagebox.askokcancel("Quit", "Are you sure you want to close the registration window?"):
            self.window.deiconify()  

    def start_attendance(self):
        end_button=1
        faceDetect=cv2.CascadeClassifier("haarcascade_frontalface_default.xml");
        path = 'C:/Users/hp/Desktop/pbl'
        images = []
        classNames = []
        roll = []
        myList = os.listdir(path)
        print(myList)
        for cl in myList:
            curImg = cv2.imread(os.path.join(path, cl))
            images.append(curImg)

            name = os.path.splitext(cl)[0]
            classNames.append(name)
            roll_n = name.split('.')[0]
            roll.append(roll_n)

        print(classNames)
        print(roll)


        def findEncodings(images):
            encodeList = []

            for img in images:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                face_encodings = face_recognition.face_encodings(img)
                if len(face_encodings) > 0:
                    encode = face_encodings[0]
                    encodeList.append(encode)
                else:
                    print("No face detected in the image.")

            return encodeList




        encodeListKnown = findEncodings(images)



        student = classNames.copy()

        def mark_present(students,lecture):
            sub = lecture
            attendance_file = f'attendance/{sub}.xlsx'

            if not os.path.exists(attendance_file):
                workbook = Workbook()
                sheet = workbook.active
                sheet.cell(row=1, column=1, value='Student Name')
                sheet.cell(row=1, column=2, value='Timestamp')

                count = 2
                for name in students:
                    sheet.cell(row=count, column=1, value=name)
                    count += 1

                workbook.save(attendance_file)
                print(f"Creating Spreadsheet with Title: {sub}")

            workbook = load_workbook(attendance_file)
            sheet = workbook.active

            sheet.cell(row=1, column=sheet.max_column + 1, value='Timestamp')

            for i, name in enumerate(students, start=2):
                if name in output:
                    sheet.cell(row=i, column=sheet.max_column, value='P')
                else:
                    sheet.cell(row=i, column=sheet.max_column, value='A')


            workbook.save(attendance_file)
            print("Total Present Students:",len(output))

        
        import time
        from datetime import datetime, timedelta

        present =[]


        def start_camera(duration):
            
            cam = cv2.VideoCapture(0)  
            
            start_time = time.time()
            end_time = start_time + duration_minutes * 60

            while time.time() < end_time:
                ret,img=cam.read()
                label = test(
                        image=img,
                        model_dir='C:\\Users\\hp\\Silent-Face-Anti-Spoofing-master\\resources\\anti_spoof_models',
                        device_id=0
                        )
                if label ==1:
                    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                    faces=faceDetect.detectMultiScale(gray,1.3,5)
                    for(x,y,w,h) in faces:
                        cv2.rectangle(img,(x,y),(x+w,y+h),(0,140,255),2)
                        roi_gray = gray[y:y+h, x:x+w]
                        roi_color = img[y:y+h, x:x+w]
                        face = img[y:y+h, x:x+w]
                        facesCurFrame = face_recognition.face_locations(face)
                        encodesCurFrame = face_recognition.face_encodings(face, facesCurFrame)
                        for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
                            matches = face_recognition.compare_faces(encodeListKnown , encodeFace)
                            name=""
                            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
                            matchIndex = np.argmin(faceDis)

                            if matches[matchIndex]:
                                name = classNames[matchIndex]
                            if name in classNames:
                                if name in student:
                                    output.append(name)
                                    print (name)
                                    student.remove(name)


                            else:
                                print("UNRECOGNIZED PERSON")

                cv2.imshow('Face',img)
                if cv2.waitKey(10)==ord('q') :
                    end_button=0
                    break
            end_button=0
            cam.release()
            cv2.destroyAllWindows()
            


        



        
        import datetime
        import time
        import pandas as pd

        
        output = []

        def check_schedule(schedule_df):
            now = datetime.datetime.now()
            day = now.weekday()
            hour = now.hour
            minute = now.minute

            matching_schedule = schedule_df[
                (schedule_df['Day'] == day) &
                (schedule_df['Hour'] == hour) &
                (schedule_df['Minute'] == minute)
            ]

            return not matching_schedule.empty


        schedule_df = pd.read_excel('schedule.xlsx')
        end_button=1
        while (end_button!=0):
            if check_schedule(schedule_df):
                matching_schedule = schedule_df[
                    (schedule_df['Day'] == datetime.datetime.now().weekday()) &
                    (schedule_df['Hour'] == datetime.datetime.now().hour) &
                    (schedule_df['Minute'] == datetime.datetime.now().minute)
                ]

                duration_minutes = matching_schedule['Duration'].values[0]
                lecture_name = matching_schedule['Lecture Name'].values[0]
                print(lecture_name)
                start_camera(duration_minutes)
                print(output)
                mark_present(classNames,lecture_name)
                student = classNames.copy()


            time.sleep(30) 

window = tk.Tk()
app = WelcomeScreen(window)
window.mainloop()

['23.Shresayas.jpg', '40.Atharva.jpg', '40sudhanshu.jpg', '54sanket.jpg', '74.Ganesh.jpg', 'shivam.jpg', 'sudhanshu.jpg']
['23.Shresayas', '40.Atharva', '40sudhanshu', '54sanket', '74.Ganesh', 'shivam', 'sudhanshu']
['23', '40', '40sudhanshu', '54sanket', '74', 'shivam', 'sudhanshu']


In [3]:
!pip install opencv-python


  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
